In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM, Convolution1D, Flatten, Dropout,MaxPooling1D
# from keras.layers.convolution import 
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import TensorBoard
import ast
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import keras as kr
from keras.callbacks import ModelCheckpoint
from sklearn.utils import class_weight
kr.__version__

Using TensorFlow backend.


In [ ]:
base_dir_tr='training/'
base_dir_vl='validation/'
weight_log='weight_logs/'

In [ ]:
top_words = 20000
dtr_1=pd.read_csv(base_dir_tr+'data_manipulation_2_2_train.csv',encoding='ISO-8859-1')
dtv_1=pd.read_csv(base_dir_vl+'data_manipulation_2_2_valid.csv',encoding='ISO-8859-1')

In [ ]:
Xsd=dtr_1.rank_short_des.as_matrix()
Xsd=np.array([ast.literal_eval(cd) for cd in Xsd])

Xti=dtr_1.rank_title.as_matrix()
Xti=np.array([ast.literal_eval(cd) for cd in Xti])

ycl=dtr_1.clari

In [ ]:
Xsd_v=dtv_1.rank_short_des.as_matrix()
Xsd_v=np.array([ast.literal_eval(cd) for cd in Xsd_v])

Xti_v=dtv_1.rank_title.as_matrix()
Xti_v=np.array([ast.literal_eval(cd) for cd in Xti_v])

In [ ]:
max_review_length_s = 150
max_review_length_t = 40
Xsd = sequence.pad_sequences(Xsd, maxlen=max_review_length_s)
Xti = sequence.pad_sequences(Xti, maxlen=max_review_length_t)

Xsd_v = sequence.pad_sequences(Xsd_v, maxlen=max_review_length_s)
Xti_v = sequence.pad_sequences(Xti_v, maxlen=max_review_length_t)

In [ ]:
X=[]
for i,j in zip(Xsd,Xti):
    X.append(list(j)+list(i))
X=np.array(X)

X_v=[]
for i,j in zip(Xsd_v,Xti_v):
    X_v.append(list(j)+list(i))
X_v=np.array(X_v)

# np.savetxt(base_dir_vl+'data_input_ycl_3_1.txt',X_v)
# np.savetxt(base_dir_vl+'data_input_ycl_3_2.txt',X_v)
# np.savetxt(base_dir_vl+'data_input_ycl_3_3.txt',X_v)
# np.savetxt(base_dir_vl+'data_input_ycl_3_4.txt',X_v)
np.savetxt(base_dir_vl+'data_input_ycl_3_5.txt',X_v)

In [ ]:
X[0]

#### yco model

In [ ]:
# model.save('model_yco_1_2_3.h5') OKE (74.05%)
ycl.value_counts()/ycl.value_counts().sum()

In [ ]:
clas_weight = class_weight.compute_class_weight('balanced', np.unique(ycl), ycl)
clas_weight

In [ ]:
X_train, X_test, ycl_train, ycl_test = train_test_split(X, ycl, test_size=0.3, random_state=21)

In [ ]:
len(X_train)

In [ ]:
#### Creating the model_1
np.random.seed(21)

# Using embedding from Keras
embedding_vecor_length = 150
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length_s+max_review_length_t))

# model.add(Convolution1D(64, 3, border_mode='same'))
# model.add(MaxPooling1D(2))
model.add(LSTM(80))
# model.add(Flatten())
model.add(Dropout(0.05))
model.add(Dense(300,activation='sigmoid'))
# model.add(Dense(300,activation='sigmoid'))
# model.add(Dense(400,activation='sigmoid'))
# model.add(Dense(500,activation='sigmoid'))
model.add(Dropout(0.05))
model.add(Dense(1,activation='sigmoid'))

model2=model

# Log to tensorboard
# tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','mse'])

# checkpoint
# filepath=weight_log+"weights_best_ycl_3_1.hdf5"
# filepath=weight_log+"weights_best_ycl_3_2.hdf5"
# filepath=weight_log+"weights_best_ycl_3_3.hdf5"
# filepath=weight_log+"weights_best_ycl_3_4.hdf5"
filepath=weight_log+"weights_best_ycl_3_5.hdf5"


checkpoint = ModelCheckpoint(filepath, monitor='val_mean_squared_error', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


In [126]:
# model.summary()
print(model.input_shape,X_train.shape)

(None, 190) (25398, 190)


In [127]:
model.fit(X_train, ycl_train, validation_data=(X_test,ycl_test),callbacks=callbacks_list, epochs=20, batch_size=100, verbose=1,class_weight = clas_weight)

Train on 25398 samples, validate on 10885 samples
Epoch 1/20
25398/25398 [==============================] - 794s - loss: 0.2042 - acc: 0.9427 - mean_squared_error: 0.0523 - val_loss: 0.1827 - val_acc: 0.9450 - val_mean_squared_error: 0.0480
Epoch 2/20
25398/25398 [==============================] - 733s - loss: 0.1620 - acc: 0.9430 - mean_squared_error: 0.0458 - val_loss: 0.1907 - val_acc: 0.9444 - val_mean_squared_error: 0.0500
Epoch 3/20
25398/25398 [==============================] - 817s - loss: 0.1239 - acc: 0.9528 - mean_squared_error: 0.0363 - val_loss: 0.2270 - val_acc: 0.9362 - val_mean_squared_error: 0.0541
Epoch 4/20
 3200/25398 [==>...........................] - ETA: 619s - loss: 0.0895 - acc: 0.9631 - mean_squared_error: 0.0265

KeyboardInterrupt: 

In [ ]:
# model2.load_weights(weight_log+"weights_best_ycl_3_1.hdf5")
# model2.load_weights(weight_log+"weights_best_ycl_3_2.hdf5")
# model2.load_weights(weight_log+"weights_best_ycl_3_3.hdf5")
# model2.load_weights(weight_log+"weights_best_ycl_3_4.hdf5")
model2.load_weights(weight_log+"weights_best_ycl_3_5.hdf5")

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','mse'])

In [ ]:
#### Evaluation on the test set
ycl_predc=model2.predict_classes(X_test)
ycl_predp=model2.predict(X_test)

ycl_predc=[i[0] for i in ycl_predc]
ycl_predp=[i[0] for i in ycl_predp]

scores = model2.evaluate(X_test, ycl_test, verbose=0)
mse_c=mse(ycl_test,ycl_predc)
mse_p=mse(ycl_test,ycl_predp)
print("Accuracy: %.2f%%" % (scores[1]*100)) 
print("MSE_class : ",mse_c)
print("MSE_prob : ",mse_p)

In [49]:
# model.save('model_ycl_2_1_2.h5') <-3 epoch 0.04628
# model2.save('model_ycl_3_1.h5') #<- 0.0468
# model2.save('model_ycl_3_2.h5') #<- 0.04676
# model2.save('model_ycl_3_3.h5') #<-0.04609
# model2.save('model_ycl_3_4.h5') #<-0.04567
model2.save('model_ycl_3_5.h5')

In [ ]:
#### model.save('model_ycl_3_3.h5')
np.random.seed(21)

# Using embedding from Keras
embedding_vecor_length = 150
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length_s+max_review_length_t))

# model.add(Convolution1D(256, 3, border_mode='same'))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
# model.add(Convolution1D(8, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.35))
model.add(Dense(200,activation='sigmoid'))
model.add(Dense(200,activation='sigmoid'))
# model.add(Dense(200,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

model2=model

# Log to tensorboard
# tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','mse'])

### Validation

In [50]:
# X_vl=np.loadtxt(base_dir_vl+'data_input_ycl_3_1.txt')
# ycl_model=load_model('model_ycl_3_1.h5')

# X_vl=np.loadtxt(base_dir_vl+'data_input_ycl_3_2.txt')
# ycl_model=load_model('model_ycl_3_2.h5')

# X_vl=np.loadtxt(base_dir_vl+'data_input_ycl_3_3.txt')
# ycl_model=load_model('model_ycl_3_3.h5')

# X_vl=np.loadtxt(base_dir_vl+'data_input_ycl_3_4.txt')
# ycl_model=load_model('model_ycl_3_4.h5')

X_vl=np.loadtxt(base_dir_vl+'data_input_ycl_3_5.txt')
ycl_model=load_model('model_ycl_3_5.h5')

In [66]:
X_v==X_vl

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ..., 
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

In [51]:
ycl_predp=ycl_model.predict(X_vl)
ycl_predp=[i[0] for i in ycl_predp]

In [52]:
with open('clarity_valid.predict','w') as fcl:
    for pp in ycl_predp:
        fcl.write(str(pp))
        fcl.write('\n')
fcl.close()